# Running on ServiceX

Some short examples at this point.

If you disconnect from the internet in the middle of a query execution, it will continue to execute inside ServiceX. You can then re-run the query, and the software should realize it was interupted, and fetch the results from the query you already made.

In [3]:
from func_adl import ObjectStream
from config import ds_xaod

## UpROOT

## ATLAS R21 XAOD

## CMS Run 1 AOD

## Common Features

There are a few things that are common to running on the `ServiceX` backend. Most, but not all, of these are a function of the front-end library. When using `func_adl` direct access to this library is mostly hidden.

* [Introduction to the `servicex` library and the up-to-date details.](https://github.com/ssl-hep/ServiceX_frontend)
* [Releases](https://github.com/ssl-hep/ServiceX_frontend/releases) of the library, which should give you an idea of what has changed most recently.
* See the [chapter on the frontend `servicex` library's features](ch-controlling-sx).

## `ServiceX` Dashboard

Lets start with the dashboard, which is not part of the front-end library. Rather, it is one of the pages served from the service itself. Here is a typical picture while a query is running: 

## Query Title

It is possible to add a title to your query. The title will appear on the `ServiceX` dashboard (the dash board is very common) - and is used no where else. This simply makes life easier if your code ends up running 20 or 30 queries to tell each one apart. The query is added to the `title` parameter of `value` method:

In [2]:
help(ObjectStream.value)

Help on function value_async in module func_adl.object_stream:

value_async(self, executor: 'Optional[Callable[[ast.AST, Optional[str]], Any]]' = None, title: 'Optional[str]' = None) -> 'Any'
    Evaluate the ObjectStream computation graph. Tracks back to the source dataset to
    understand how to evaluate the AST. It is possible to pass in an executor to override that
    behavior (used mostly for testing).
    
    Arguments:
    
        executor        A function that when called with the ast will return a future for the
                        result. If None, then uses the default executor. Normally is none
                        and the default executor specified by the `EventDatasource` is called
                        instead.
        title           Optional title to hand to the transform
    
    Returns
    
        The first element of the ObjectStream after evaluation.
    
    
    Note
    
        This is the non-blocking version - it will return a future which can


Note that the `value` and `value_async` have the same signature. See the [`async` ServiceX](sec-async) section for more information about the `async` API.

Here is what the actual query looks like and how it shows up on the ServiceX Dashboard:

In [4]:
v_jets = (ds_xaod
          .Select(lambda e: e.Jets())
          .Select(lambda jets: [j for j in jets if j.pt()/1000.0 > 30])
          .Select(lambda jets: {
              'pt': [j.pt()/1000.0 for j in jets],
          })
          .AsAwkwardArray()
          .value(title="Test Job")
)

Test Job:   0%|          | 0/9000000000.0 [00:00]

        Test Job Downloaded:   0%|          | 0/9000000000.0 [00:00]

While it is running the job name will be included in the marching ant's display:

```{image} assets/notebook_title_job_running_status.png
:alt: Notebook Marching Ants
:width: 600px
:align: center
```

And the dash board will also be labeled:

```{image} assets/dashboard_with_title.png
:alt: Dashboard with transform title
:width: 600px
:align: center
```

Unless you are on an open unauthenticated ServiceX instance, only you will be able to see the title (along with the sys-admins!).